# Analog Output with AOTask

This notebook demonstrates the `AOTask` class from `nidaqwrapper` for generating analog output signals on NI-DAQmx hardware.

**Key features:**
- Programmatic channel configuration (no NI MAX required)
- Continuous regeneration mode for looping waveforms
- Direct delegation to `nidaqmx.Task` — the hardware task is the single source of truth
- NumPy-friendly API with `(n_samples, n_channels)` format

**Architecture:** The `nidaqmx.Task` is created immediately in the constructor. All methods delegate directly to it — no intermediate state is maintained.

## Setup and Hardware Detection

First, check if `nidaqmx` is available. If not, set up a lightweight mock to allow the notebook to run.

In [ ]:
# Try to import nidaqmx, fall back to mock if unavailable
try:
    import nidaqmx
    from nidaqmx import constants
    HW_AVAILABLE = True
    print("Running with real NI-DAQmx hardware")
except ImportError:
    print("nidaqmx not available — setting up mock mode")
    from unittest.mock import MagicMock, PropertyMock
    import sys
    
    # Mock nidaqmx module
    nidaqmx = MagicMock()
    sys.modules['nidaqmx'] = nidaqmx
    
    # Mock constants
    constants = MagicMock()
    constants.AcquisitionType.CONTINUOUS = 10123
    constants.RegenerationMode.ALLOW_REGENERATION = 10097
    constants.VoltageUnits.VOLTS = 10348
    nidaqmx.constants = constants
    sys.modules['nidaqmx.constants'] = constants
    
    # Mock system with fake devices
    mock_device = MagicMock()
    mock_device.name = "SimDev1"
    mock_device.product_type = "Simulated DAQ Device"
    
    mock_system = MagicMock()
    mock_system.devices = [mock_device]
    mock_system.tasks.task_names = []
    nidaqmx.system.System.local.return_value = mock_system
    
    # Mock Task class
    def mock_task_factory(new_task_name=None):
        task = MagicMock()
        task.name = new_task_name or "unnamed_task"
        task.channel_names = []
        task.ao_channels = []
        
        # Mock add_ao_voltage_chan to track channels
        def add_channel(physical_channel, name_to_assign_to_channel=None, 
                       min_val=-10.0, max_val=10.0):
            ch = MagicMock()
            ch.physical_channel.name = physical_channel
            task.ao_channels.append(ch)
            task.channel_names.append(name_to_assign_to_channel or physical_channel)
        
        task.ao_channels.add_ao_voltage_chan = add_channel
        
        # Mock timing
        task.timing.samp_clk_rate = 10000.0
        task.timing.samp_quant_samp_per_chan = 50000
        
        # Mock write to accept data silently
        task.write = MagicMock()
        task.start = MagicMock()
        task.close = MagicMock()
        
        # Mock regeneration mode
        task._out_stream = MagicMock()
        
        return task
    
    nidaqmx.task.Task = mock_task_factory
    nidaqmx.Task = mock_task_factory
    
    HW_AVAILABLE = False
    print("Mock mode configured successfully")

import numpy as np
import matplotlib.pyplot as plt

print(f"\nHardware available: {HW_AVAILABLE}")

In [ ]:
# Import nidaqwrapper
import sys
sys.path.insert(0, '/home/user1/data/_Github/ni-daq-python/nidaqwrapper/src')

from nidaqwrapper import AOTask

print("AOTask imported successfully")

## 1. Creating an AOTask

The `AOTask` constructor creates an analog output task immediately. The underlying `nidaqmx.Task` is allocated right away.

**Parameters:**
- `task_name` (str): Unique name, must not collide with NI MAX tasks
- `sample_rate` (float): Output sample rate in Hz
- `samples_per_channel` (int, optional): Buffer size, defaults to 5 seconds (`5 * int(sample_rate)`)

In [ ]:
# Create an analog output task
task = AOTask(
    task_name="signal_generator",
    sample_rate=10000,  # 10 kHz
    samples_per_channel=50000  # 5 seconds at 10 kHz
)

print(f"Created task: {task.task_name}")
print(f"Sample rate: {task.sample_rate} Hz")
print(f"Buffer size: {task.samples_per_channel} samples/channel")
print(f"Detected devices: {task.device_list}")

## 2. Adding Analog Output Channels

Channels are added via `add_channel()`, which delegates directly to `task.ao_channels.add_ao_voltage_chan()`.

**Parameters:**
- `channel_name` (str): Logical name for the channel
- `device_ind` (int): Index into `device_list`
- `channel_ind` (int): AO channel number (e.g., 0 for `ao0`)
- `min_val` (float): Minimum voltage, default -10.0
- `max_val` (float): Maximum voltage, default 10.0

In [ ]:
# Add a voltage output channel
task.add_channel(
    channel_name="ao_sine",
    device_ind=0,
    channel_ind=0,
    min_val=-5.0,
    max_val=5.0
)

print(f"Channels configured: {task.channel_list}")
print(f"Number of channels: {task.number_of_ch}")

## 3. Configuring Timing with `start()`

The `start()` method configures sample-clock timing, enables regeneration mode, and optionally starts the task.

**Parameters:**
- `start_task` (bool): If True, start the task immediately. Default is False.

**Notes:**
- Validates that the driver accepts the requested sample rate (some devices only support discrete rates)
- Enables `ALLOW_REGENERATION` mode for continuous looping
- If `start_task=False`, you must start manually or use a hardware trigger

In [ ]:
# Configure timing (don't start yet)
task.start(start_task=False)

print("Timing configured successfully")
print(f"Sample mode: {task.sample_mode}")

## 4. Generating Signals

The `generate()` method writes signal data to the output buffer. Data format is `(n_samples, n_channels)` for the public API.

**Internally:**
- 2D arrays are transposed to `(n_channels, n_samples)` for nidaqmx
- `np.ascontiguousarray()` ensures C-contiguous memory layout
- Calls `task.write(data, auto_start=True)` — the task starts automatically if not already running

**Supported formats:**
- `(n_samples,)` — single-channel 1D array
- `(n_samples, 1)` — single-channel 2D array
- `(n_samples, n_channels)` — multi-channel 2D array

In [ ]:
# Generate a 1 Hz sine wave at 10 kHz sample rate (5 seconds)
sample_rate = 10000
duration = 5.0
frequency = 1.0
amplitude = 4.0  # Within the ±5V range we configured

t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
sine_wave = amplitude * np.sin(2 * np.pi * frequency * t)

print(f"Generated signal shape: {sine_wave.shape}")
print(f"Signal range: [{sine_wave.min():.2f}, {sine_wave.max():.2f}] V")

# Plot the first second
plt.figure(figsize=(10, 4))
plt.plot(t[:sample_rate], sine_wave[:sample_rate])
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('1 Hz Sine Wave (First Second)')
plt.grid(True)
plt.show()

In [ ]:
# Write the signal to the output task
task.generate(sine_wave)

print("Signal written to output buffer")
if HW_AVAILABLE:
    print("Analog output is now generating continuously (regeneration mode)")
else:
    print("(Mock mode — no actual hardware output)")

## 5. Multi-Channel Output

For multi-channel output, use a 2D array with shape `(n_samples, n_channels)`. Each column represents one channel.

In [ ]:
# Clean up the single-channel task
task.clear_task()

# Create a new task with two channels
task_multi = AOTask(
    task_name="dual_signal_gen",
    sample_rate=10000,
    samples_per_channel=10000  # 1 second buffer
)

# Add two channels
task_multi.add_channel("ao_sine", device_ind=0, channel_ind=0, min_val=-5.0, max_val=5.0)
task_multi.add_channel("ao_cosine", device_ind=0, channel_ind=1, min_val=-5.0, max_val=5.0)

task_multi.start(start_task=False)

print(f"Multi-channel task created with {task_multi.number_of_ch} channels")
print(f"Channels: {task_multi.channel_list}")

In [ ]:
# Generate sine and cosine waves (90° phase shift)
duration = 1.0
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
frequency = 2.0
amplitude = 3.0

sine = amplitude * np.sin(2 * np.pi * frequency * t)
cosine = amplitude * np.cos(2 * np.pi * frequency * t)

# Stack into (n_samples, n_channels) format
multi_channel_signal = np.column_stack([sine, cosine])

print(f"Multi-channel signal shape: {multi_channel_signal.shape}")

# Plot both signals
plt.figure(figsize=(10, 4))
plt.plot(t[:500], sine[:500], label='Channel 0 (sine)')
plt.plot(t[:500], cosine[:500], label='Channel 1 (cosine)')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Dual-Channel Output: Sine and Cosine (First 0.05s)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Write the multi-channel signal
task_multi.generate(multi_channel_signal)

print("Multi-channel signal written to output buffer")
if HW_AVAILABLE:
    print("Both channels generating continuously")
else:
    print("(Mock mode — no actual hardware output)")

## 6. Context Manager Usage

`AOTask` supports the context manager protocol (`with` statement). This ensures automatic cleanup even if an exception occurs.

In [ ]:
# Context manager automatically calls clear_task() on exit
with AOTask("context_task", sample_rate=10000) as ctx_task:
    ctx_task.add_channel("ao_ctx", device_ind=0, channel_ind=0)
    ctx_task.start()
    
    # Generate a simple ramp signal
    ramp = np.linspace(-2.0, 2.0, 50000)
    ctx_task.generate(ramp)
    
    print(f"Task active: {ctx_task.task is not None}")

# Task is automatically cleaned up here
print("Context manager exited — task cleaned up automatically")

## 7. Manual Cleanup

The `clear_task()` method closes the nidaqmx task and releases hardware resources. Safe to call multiple times.

**Notes:**
- Always call `clear_task()` when done with a task (or use context manager)
- For externally-provided tasks (created via `from_task()`), this method does NOT close the task — a warning is issued instead

In [ ]:
# Clean up the multi-channel task
task_multi.clear_task()

print("Multi-channel task cleaned up")

## 8. TOML Configuration

Tasks can be saved to human-readable TOML files and loaded back. This enables hardware portability — device names are aliased, so changing chassis enumeration only requires editing the config file.

**Methods:**
- `save_config(path)`: Write task configuration to TOML
- `from_config(path)`: Class method to create a task from TOML (channels added, not started)

In [ ]:
# Create a task and save its configuration
config_task = AOTask("config_demo", sample_rate=5000, samples_per_channel=25000)
config_task.add_channel("sig_ch0", device_ind=0, channel_ind=0, min_val=-10.0, max_val=10.0)
config_task.add_channel("sig_ch1", device_ind=0, channel_ind=1, min_val=-5.0, max_val=5.0)

config_path = "/tmp/ao_task_config.toml"
config_task.save_config(config_path)

print(f"Configuration saved to {config_path}")

# Read and display the TOML file
with open(config_path, 'r') as f:
    print("\nTOML content:")
    print(f.read())

config_task.clear_task()

In [ ]:
# Load the configuration back
loaded_task = AOTask.from_config(config_path)

print(f"Task loaded from config: {loaded_task.task_name}")
print(f"Channels: {loaded_task.channel_list}")
print(f"Sample rate: {loaded_task.sample_rate} Hz")

# The task is ready to start and generate
loaded_task.start()
print("Task started successfully")

loaded_task.clear_task()

## 9. Advanced: Wrapping External nidaqmx Tasks

The `from_task()` class method wraps a pre-created `nidaqmx.Task`. This provides an escape hatch for advanced users who need to configure properties not exposed by the wrapper API.

**Important:**
- The task must already have AO channels configured
- `add_channel()` and `start()` are blocked (raise `RuntimeError`)
- `clear_task()` does NOT close the task — caller is responsible
- Useful for custom trigger configurations, advanced timing, etc.

In [ ]:
if HW_AVAILABLE:
    # Create a raw nidaqmx task
    raw_task = nidaqmx.Task("raw_ao_task")
    raw_task.ao_channels.add_ao_voltage_chan("Dev1/ao0", min_val=-5.0, max_val=5.0)
    raw_task.timing.cfg_samp_clk_timing(rate=10000, sample_mode=constants.AcquisitionType.CONTINUOUS)
    
    # Wrap it with AOTask
    wrapped = AOTask.from_task(raw_task)
    
    print(f"Wrapped external task: {wrapped.task_name}")
    print(f"Channels: {wrapped.channel_list}")
    print(f"Sample rate: {wrapped.sample_rate} Hz")
    
    # Can use generate() but not add_channel() or start()
    signal = np.sin(2 * np.pi * np.linspace(0, 5, 50000))
    wrapped.generate(signal)
    print("Signal generated via wrapped task")
    
    # Cleanup: must close the raw task manually
    wrapped.clear_task()  # Issues warning, does not close
    raw_task.close()  # Caller's responsibility
    print("Raw task closed manually")
else:
    print("from_task() example skipped in mock mode")

## Summary

The `AOTask` class provides a clean, NumPy-friendly interface for analog output generation:

1. **Create:** `AOTask(task_name, sample_rate, samples_per_channel)`
2. **Configure channels:** `add_channel(name, device_ind, channel_ind, min_val, max_val)`
3. **Configure timing:** `start(start_task=False)` — sets up continuous regeneration
4. **Generate signals:** `generate(signal)` — writes `(n_samples, n_channels)` data, auto-starts
5. **Cleanup:** `clear_task()` or use context manager

**Key patterns:**
- Direct delegation: `nidaqmx.Task` is the single source of truth
- Data format: public API uses `(n_samples, n_channels)`, transposed internally
- Regeneration mode: signals loop continuously until task is stopped
- TOML config: save/load for hardware portability
- External tasks: wrap pre-created `nidaqmx.Task` objects with `from_task()`

**Next steps:**
- See `03_digital_io.ipynb` for digital input/output
- See `04_handler.ipynb` for the high-level `DAQHandler` interface
- See `05_multi_handler.ipynb` for synchronized multi-task operations